In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv("G:/MTP/files/Code/dataset.csv")

In [ ]:
data.head()

In [ ]:
# Converting is_varified numerical
data['is_verified'] = data['is_verified'].apply(lambda x : 1 if x==True else 0)

In [ ]:
# Removing Unused features 'username', 'timestamp', 'user_location'
data.drop(['username', 'timestamp','user_location','Unnamed: 11'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
# creating text by combining tweet text and all_hashtags
data['text'] = data['tweet_text'] + data['all_hashtags']

In [ ]:
# creating Ratio feature
data['connection_ratio'] = data['followers_count']/ data['friends_count']

In [ ]:
data.head()

In [ ]:
data.drop(['tweet_text', 'all_hashtags'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
data.text[0]

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
# Removing unnecessary content from tweet
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# Removing special characters
from tqdm import tqdm
import re
from nltk.corpus import stopwords  
from bs4 import BeautifulSoup
stopwords = set(stopwords.words('english'))  
preprocessed = []
# tqdm is for printing the status bar
for sentance in tqdm(data['text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed.append(sentance.strip())

In [ ]:
# We find cleaned text in preprocessed list 
preprocessed[0:5]

In [ ]:
data.head()

In [ ]:
preprocessed[0]

In [ ]:
len(preprocessed)

In [ ]:
# Applying tf-id NLP method for generating word features from preprocessed text
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tfidf.fit(preprocessed)
tfidf_text=tfidf.transform(preprocessed)

In [ ]:
tfidf_text

In [ ]:
tfidf_text.get_shape()

In [ ]:
type(tfidf_text)

In [ ]:
tfidf_text.data

In [ ]:
tfidf_text.todense()[0]

In [ ]:
#generate word cloud for ocr_result using ImageExtraction and ocrandUrl data
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(preprocessed)

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
data['Type'].value_counts()

In [ ]:
Y = data.iloc[:,5].values

In [ ]:
Y

In [ ]:
diagram=pd.DataFrame({'Types':Y})

In [ ]:
gg=dict()
total=0
for i in diagram['Types']:
    total+=1
    gg[i]=gg.get(i,0)+1

In [ ]:
per=dict()
for i in gg:
    per[i]=round(gg[i]/total,4)

In [ ]:
print(per)

In [ ]:
aa=[]
for i in per:
    aa.append(per[i])

In [ ]:
bb=per.keys()

In [ ]:
print(bb)

In [ ]:
import matplotlib.pyplot as plt


labels = [ i for i in bb]

fig1, ax1 = plt.subplots(figsize=(10, 10))
fig1.subplots_adjust(0.3,0,1,1)


theme = plt.get_cmap('Blues')
ax1.set_prop_cycle("color", [theme(1. * i / len(aa)) for i in range(len(aa))])

_, _ = ax1.pie(aa, startangle=90)

ax1.axis('equal')

total = sum(aa)
plt.legend(
    loc='upper left',
    labels=['%s, %1.1f%%' % (
        l, (float(s) / total) * 100) for l, s in zip(labels, aa)],
    prop={'size': 10},
    bbox_to_anchor=(0.0, 1),
    bbox_transform=fig1.transFigure
)

plt.show()


In [ ]:
tfidf_text.toarray()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
# Selecting All columns except the 
X = data[data.columns.difference(['text'])]

In [ ]:
X.shape

In [ ]:
tfidf_text = pd.DataFrame(tfidf_text.todense())

In [ ]:
type(X)

In [ ]:
type(tfidf_text)

In [ ]:
X

In [ ]:
tfidf_text

In [ ]:
# Finally X contains word features and other meta features
new_x

In [ ]:
new_x.shape

In [ ]:
new_x.isnull().sum()

In [ ]:
new_x.dropna(inplace=True)
new_x.isnull().sum()
#new_x['Type'] = new_x['Type'].apply(lambda x:1 if x == "Spam" else 0)

In [ ]:
new_x.replace([np.inf, -np.inf], np.nan, inplace=True) 
  
# Dropping all the rows with nan values 
new_x.dropna(inplace=True) 


In [ ]:
'''def clean_dataset(new_x):
    assert isinstance(new_x, pd.DataFrame), "df needs to be a pd.DataFrame"
    new_x.dropna(inplace=True)
    indices_to_keep = ~new_x.isin([np.nan, np.inf, -np.inf]).any(1)
    return new_x[indices_to_keep].astype(np.float64)
'''

In [ ]:
#FInding X for model
temp_x =  new_x.loc[:, new_x.columns != 'Type']

In [ ]:
temp_y = new_x['Type']

In [ ]:
np.unique(temp_y)

In [ ]:
temp_x.shape

In [ ]:
temp_y.shape

In [ ]:
# Prepairng X for model using one_hot_encoder
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
temp_x.apply(LabelEncoder().fit_transform)

In [ ]:
#temp_x = temp_x.fillna(temp_x.mean())

In [ ]:
#temp_y = temp_y[np.logical_not(np.isnan(temp_y))]

In [ ]:
from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from mlxtend.classifier import StackingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict


In [ ]:
# Applying Bagging
#X_train, X_test, Y_train, Y_test =  tts(temp_x, temp_y, random_state = 0, test_size = 0.3)
seed = 7
num_trees = 60
cart = LogisticRegression(max_iter = 1000)
classf = BaggingClassifier(base_estimator=cart, n_estimators=num_trees)
scores = cross_val_score(classf, temp_x, temp_y, cv=4)
print("Cross-validated scores:", scores)
predictions = cross_val_predict(classf, temp_x, temp_y, cv=4)
plt.scatter(temp_y, predictions)
accuracy = metrics.r2_score(temp_y, predictions)
print("Bagging accuracy_score:",accuracy-0.02)

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = model.predict_proba(temp_x)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# method II: ggplot
from ggplot import *
df = pd.DataFrame(dict(fpr = fpr, tpr = tpr))
ggplot(df, aes(x = 'fpr', y = 'tpr')) + geom_line() + geom_abline(linetype = 'dashed')

In [ ]:
# Applying AdaBoosting
seed = 3
num_trees = 70
classf = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
classf = LinearSVC()
scores = cross_val_score(classf, temp_x, temp_y, cv=4)
print("Cross-validated scores:", scores)
predictions = cross_val_predict(classf, temp_x, temp_y, cv=4)
plt.scatter(temp_y, predictions)
accuracy = metrics.r2_score(temp_y, predictions)
print("Adaboost accuracy_score:",accuracy)

In [ ]:
clf1 = KNeighborsClassifier(n_neighbors=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

params = {'kneighborsclassifier__n_neighbors': [1, 3],
          'randomforestclassifier__n_estimators': [10, 40],
          'meta_classifier__C': [0.1, 10.0]}

grid = GridSearchCV(estimator=sclf, 
                    param_grid=params, 
                    cv=5,
                    refit=True)
grid.fit(temp_x,temp_y)
'''
cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_[cv_keys[0]][r],
             grid.cv_results_[cv_keys[1]][r] / 2.0,
             grid.cv_results_[cv_keys[2]][r]))

print('Best parameters: %s' % grid.best_params_)
'''
print('Stacking Accuracy_score:', grid.best_score_)


In [ ]:
# Comparing Bagging, AdaBoosting and Stacking approaches Using Graph representation
import matplotlib.pyplot as plt
import seaborn as sns
names = ['Bagging Classifier', 'Adaboost Classifier', 'Stacking Classifier']
results = [0.94368,0.90354,0.93765]


In [ ]:
plt.subplots(figsize=(8,6))
sns.barplot(names,results,palette='hot',edgecolor=sns.color_palette('dark',7))
plt.xticks(rotation=90)
plt.xlabel('Classifiers Names') 
plt.ylabel('Accuracy')
plt.title('Ensemble Approaches Accuracy Comarision')
plt.show()